In [5]:
import openai
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT


CHAT_COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')


prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
response = openai.ChatCompletion.create(
  engine=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])


### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= CURRENT_DATE - INTERVAL '3 months'
        GROUP BY sp.employee_id
        HAVING COUNT(sp.employee_id) > 10
    )
)


In [8]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

response = openai.ChatCompletion.create(
  engine=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])


This code is a SQL query that retrieves the names of departments that have more than 10 employees who have received salary payments in the last 3 months. 

Here is a breakdown of the code:

1. The query starts with the SELECT statement, which specifies that we want to retrieve the "name" column from the "Department" table.

2. The FROM clause specifies that we are selecting from the "Department" table.

3. The JOIN keyword is used to combine the "Department" table with the "Employee" table. The ON clause specifies the condition for the join, which is that the "id" column in the "Department" table must match the "department_id" column in the "Employee" table.

4. The WHERE clause filters the results based on a condition. In this case, it checks if the "id" column in the "Employee" table is present in the subquery.

5. The subquery selects the "employee_id" column from the "Salary_Payments" table where the "date" column is greater than the current date minus 3 months.

6. The GROUP BY cl